# Avatar the Last Air Bender 
## *Sentiment Analysis of Characters, plot over time*

## Part Two: Use Vader and NLTK to Build an Emotion to Line Mapping of Episodes from Transcript text

First, we do our imports and define some helper functions.

In [1]:
import json
# python nlp toolkit
import nltk
# vader sentiment analyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# opens json file saved in part 1 as a py dictionary
def open_json_as_dict():
  with open('data/avatar-episodes.json') as f:
    data = json.load(f)
  return data

# creates a line to line mapping of vader scores for one episode
def create_vader_mapping(episode):
  sid = SentimentIntensityAnalyzer()
  vaderLines = [ [l[0], sid.polarity_scores(l[1])] for l in episode['lines'] ]
  vaderMapping = {"title": episode['title'], "vaderLines": vaderLines}
  return vaderMapping

# dump a python dictionary object to a json file
def save_vader_mapping_as_json(vaderDict):
  with open('avatar-episodes-vader-scores.json', 'w') as f:
    json.dump(vaderDict, f)

Now let's define a function to loop through all the episodes and create a new array with Vader scores.

In [23]:
def convert_episodes_to_vader_array(episodes):
  result = []
  for episode in episodes:
    vader = create_vader_mapping(episode)
    result.append(vader)
  return result

Finally time to do things! 

In [24]:
# get the data
data = open_json_as_dict()

# convert the episode lines to vader representation
vaderArray = convert_episodes_to_vader_array(data['episodes'])

# create the python dictionary
vaderMap = {
    'episodes': vaderArray
  }

# finally, save the mapping to json
save_vader_mapping_as_json(vaderMap)

### Now we have a json representation of the vader sentiment scores for every line in the Avatar Series.
We'll use that later, but for now there's a few more chunks of data we want to gather.

Let's use the lexicon from http://sentiment.nrc.ca/lexicons-for-research/ to create a method for determining emotion of text. We won't go crazy with this, but we'll try to get some okay emotional data.

Do imports and create helper functions

In [77]:
# nltk stop words to remove from our text
from nltk.corpus import stopwords
# nltk tokenizers
from nltk.tokenize import sent_tokenize, word_tokenize
# nltk lemmatizer
lemma = nltk.wordnet.WordNetLemmatizer()

# opens and returns the NRC lexicon dictionary as a list
def open_lexicon_as_list():
  result = []
  with open('data/NRC-Emotion-Lexicon.txt', 'r') as f:
      for l in f:
        result.append(l)
  return result

# simple fix for contractions
def replace_contractions(line):
  line = line.replace("'s", '')
  line = line.replace("'m", " am")
  line = line.replace("'d", " had")
  line = line.replace("'re", " are")
  line = line.replace("n't", " not")
  line = line.replace("'ve", " have")
  line = line.replace("'ll", " will")
  line = line.replace("'", '')
  return line

# use nltk to get the lemma of the word
# helps to get better matching with lexicon
def lemmalize(word):
  return lemma.lemmatize(word)

# returns nltk tokenized line after some formatting
def create_tokens_from_line(line):
  line = replace_contractions(line.lower())
  tokens = word_tokenize(line)
  noWords = ['.',',','?',':',';','!','...','[',']','``']
  stopWords = set(stopwords.words('english'))
  noWords.extend(stopWords)
  filtered = [lemmalize(w) for w in tokens if not w in noWords]
  return filtered

# dump a python dictionary object to a json file
def save_emotion_mapping_as_json(emotionDict):
  with open('data/avatar-episodes-nrc-emotion.json', 'w') as f:
    json.dump(emotionDict, f)

Define a function to determine the emotion from a particular line of text. 
This is our main workhorse function. We:
* create an emotion map
* loop through every word in a line and look up its emotion
* increment the map accordingly. 
* look at what emotion has the highest rating. 
* If all are still zero, "none" is returned. 

In [78]:
def emotion_from_episode_line(line, lexicon):
  tokens = create_tokens_from_line(line)

  # possible emotions according to NRC lexicon used
  emotionMap = {
      "anger": 0,
      "anticipation": 0,
      "disgust": 0,
      "fear": 0,
      "joy": 0,
      "negative": 0,
      "positive": 0,
      "sadness": 0,
      "surprise": 0,
      "trust": 0
  }
  
  # increment emotions for each word that maps to that emotion
  wordLookUp = []
  for word in tokens:
    mission = 10
    for l in lexicon:
        if l[0:len(word) + 1].strip() == word:
            mission -= 1
            wordLookUp.append(l.split('\t')[2].strip())
        if mission <= 0:
            break

  if wordLookUp:
      emotionMap["anger"] += int(wordLookUp[0])
      emotionMap["anticipation"] += int(wordLookUp[1])
      emotionMap["disgust"] += int(wordLookUp[2])
      emotionMap["fear"] += int(wordLookUp[3])
      emotionMap["joy"] += int(wordLookUp[4])
      emotionMap["negative"] += int(wordLookUp[5])
      emotionMap["positive"] += int(wordLookUp[6])
      emotionMap["sadness"] += int(wordLookUp[7])
      emotionMap["surprise"] += int(wordLookUp[8])
      emotionMap["trust"] += int(wordLookUp[9])

  # return the emotion which maps most closely to the line
  maxEmotion = 0
  emotion = 'none'
  for entry in emotionMap.items():
    if entry[1] > maxEmotion:
        maxEmotion = entry[1]
        emotion = entry[0]
        
  return emotion

Now let's create a function to loop through all episodes and create a new array with emotions.

In [79]:
def convert_episodes_to_emotion_array(episodes, lexicon):
  for episode in episodes:
    lines = episode['lines']
    for line in lines:
      print('converting line {' + line[1][0:8] + '} from ' + episode['title'] + '...' )
      emotion = emotion_from_episode_line(line[1], lexicon)
      line[1] = emotion
        
  return episodes

... and now we use our new functions to create a new json file with emotions per line per episode. 

In [80]:
# reload the episodes data
data = open_json_as_dict()

# open lexicon file
lexicon = open_lexicon_as_list()

# convert the episode lines to an emotional representation
emotionArray = convert_episodes_to_emotion_array(data['episodes'], lexicon)

# create the python dictionary
emoteArray = {
    'episodes': emotionArray
    }

# finally, save the mapping to json
save_emotion_mapping_as_json(emoteArray)


converting line {Water. E} from The Boy in the Iceberg...
converting line {It's not} from The Boy in the Iceberg...
converting line {[Happily} from The Boy in the Iceberg...
converting line {[Close-u} from The Boy in the Iceberg...
converting line {[Struggl} from The Boy in the Iceberg...
converting line {[Exclaim} from The Boy in the Iceberg...
converting line {Why is i} from The Boy in the Iceberg...
converting line {[Cut to } from The Boy in the Iceberg...
converting line {Yeah, ye} from The Boy in the Iceberg...
converting line {[Cut to } from The Boy in the Iceberg...
converting line {Watch ou} from The Boy in the Iceberg...
converting line {[A touch} from The Boy in the Iceberg...
converting line {You don'} from The Boy in the Iceberg...
converting line {[Indigna} from The Boy in the Iceberg...
converting line {[Annoyed} from The Boy in the Iceberg...
converting line {You are } from The Boy in the Iceberg...
converting line {Ever sin} from The Boy in the Iceberg...
converting lin

converting line {[In full} from The Boy in the Iceberg...
converting line {Think ab} from The Boy in the Iceberg...
converting line {A hundre} from The Boy in the Iceberg...
converting line {[Squats } from The Boy in the Iceberg...
converting line {[Cheerfu} from The Boy in the Iceberg...
converting line {[Smiling} from The Boy in the Iceberg...
converting line {Aang, le} from The Boy in the Iceberg...
converting line {Huh?} from The Boy in the Iceberg...
converting line {What's t} from The Boy in the Iceberg...
converting line {Uh oh ..} from The Boy in the Iceberg...
converting line {Aaah!} from The Boy in the Iceberg...
converting line {The last} from The Boy in the Iceberg...
converting line {Water. E} from The Avatar Returns...
converting line {[Joyfull} from The Avatar Returns...
converting line {[Angrily} from The Avatar Returns...
converting line {[Protest} from The Avatar Returns...
converting line {[Sheepis} from The Avatar Returns...
converting line {[Worried} from The Avata

converting line {[Iroh an} from The Southern Air Temple...
converting line {[With di} from The Southern Air Temple...
converting line {[Smirks.} from The Southern Air Temple...
converting line {[Bows ba} from The Southern Air Temple...
converting line {The Fire} from The Southern Air Temple...
converting line {Our ship} from The Southern Air Temple...
converting line {That's q} from The Southern Air Temple...
converting line {Yes. [He} from The Southern Air Temple...
converting line {[Iroh's } from The Southern Air Temple...
converting line {[Looks a} from The Southern Air Temple...
converting line {[In disb} from The Southern Air Temple...
converting line {[He aver} from The Southern Air Temple...
converting line {Prince Z} from The Southern Air Temple...
converting line {[Annoyed} from The Southern Air Temple...
converting line {[Accusin} from The Southern Air Temple...
converting line {[Surpris} from The Southern Air Temple...
converting line {[Shocked} from The Southern Air Temple.

converting line {[Happily} from The Southern Air Temple...
converting line {[Worried} from The Southern Air Temple...
converting line {Send wor} from The Southern Air Temple...
converting line {[Alarmed} from The Southern Air Temple...
converting line {Aaaaahh!} from The Southern Air Temple...
converting line {[Shoutin} from The Southern Air Temple...
converting line {[Shoutin} from The Southern Air Temple...
converting line {Oh no! I} from The Southern Air Temple...
converting line {[Holds o} from The Southern Air Temple...
converting line {Remember} from The Southern Air Temple...
converting line {[Close-u} from The Southern Air Temple...
converting line {[Confide} from The Southern Air Temple...
converting line {Basics, } from The Southern Air Temple...
converting line {[Challen} from The Southern Air Temple...
converting line {That's i} from The Southern Air Temple...
converting line {[Anger c} from The Southern Air Temple...
converting line {No, Prin} from The Southern Air Temple.

converting line {[Admirin} from The Warriors of Kyoshi...
converting line {[Holding} from The Warriors of Kyoshi...
converting line {[Turns a} from The Warriors of Kyoshi...
converting line {[Holding} from The Warriors of Kyoshi...
converting line {[Slightl} from The Warriors of Kyoshi...
converting line {I promis} from The Warriors of Kyoshi...
converting line {[Resumed} from The Warriors of Kyoshi...
converting line {[Disappo} from The Warriors of Kyoshi...
converting line {[Annoyed} from The Warriors of Kyoshi...
converting line {[Central} from The Warriors of Kyoshi...
converting line {[Stompin} from The Warriors of Kyoshi...
converting line {[Slightl} from The Warriors of Kyoshi...
converting line {[Enthusi} from The Warriors of Kyoshi...
converting line {[Sarcast} from The Warriors of Kyoshi...
converting line {It didn'} from The Warriors of Kyoshi...
converting line {[Close-u} from The Warriors of Kyoshi...
converting line {Jealous } from The Warriors of Kyoshi...
converting lin

...

converting line {Not bad,} from Sozin's Comet, Part 1: The Phoenix King...
converting line {[Amazed.} from Sozin's Comet, Part 1: The Phoenix King...
converting line {Try and } from Sozin's Comet, Part 1: The Phoenix King...
converting line {[Reveals} from Sozin's Comet, Part 1: The Phoenix King...
converting line {Is that } from Sozin's Comet, Part 1: The Phoenix King...
converting line {[Defensi} from Sozin's Comet, Part 1: The Phoenix King...
converting line {Suki, we} from Sozin's Comet, Part 1: The Phoenix King...
converting line {[Sitting} from Sozin's Comet, Part 1: The Phoenix King...
converting line {But it d} from Sozin's Comet, Part 1: The Phoenix King...
converting line {[Peeking} from Sozin's Comet, Part 1: The Phoenix King...
converting line {Teaching} from Sozin's Comet, Part 1: The Phoenix King...
converting line {What hap} from Sozin's Comet, Part 1: The Phoenix King...
converting line {[While r} from Sozin's Comet, Part 1: The Phoenix King...
converting line {Get a gr

converting line {[Walks o} from Sozin's Comet, Part 1: The Phoenix King...
converting line {Look, th} from Sozin's Comet, Part 1: The Phoenix King...
converting line {So, he w} from Sozin's Comet, Part 1: The Phoenix King...
converting line {Maybe he} from Sozin's Comet, Part 1: The Phoenix King...
converting line {I don't } from Sozin's Comet, Part 1: The Phoenix King...
converting line {I bet he} from Sozin's Comet, Part 1: The Phoenix King...
converting line {Uh-uh. H} from Sozin's Comet, Part 1: The Phoenix King...
converting line {Then wha} from Sozin's Comet, Part 1: The Phoenix King...
converting line {It's pre} from Sozin's Comet, Part 1: The Phoenix King...
converting line {But if h} from Sozin's Comet, Part 1: The Phoenix King...
converting line {[Downcas} from Sozin's Comet, Part 1: The Phoenix King...
converting line {Then he'} from Sozin's Comet, Part 1: The Phoenix King...
converting line {I'm goin} from Sozin's Comet, Part 1: The Phoenix King...
converting line {[Chantin

converting line {[Frontal} from Sozin's Comet, Part 2: The Old Masters...
converting line {The Whit} from Sozin's Comet, Part 2: The Old Masters...
converting line {[Cut clo} from Sozin's Comet, Part 2: The Old Masters...
converting line {[Camera } from Sozin's Comet, Part 2: The Old Masters...
converting line {[Frontal} from Sozin's Comet, Part 2: The Old Masters...
converting line {[Appears} from Sozin's Comet, Part 2: The Old Masters...
converting line {[Side-vi} from Sozin's Comet, Part 2: The Old Masters...
converting line {[Camera } from Sozin's Comet, Part 2: The Old Masters...
converting line {Avatar K} from Sozin's Comet, Part 2: The Old Masters...
converting line {In my da} from Sozin's Comet, Part 2: The Old Masters...
converting line {[Voice-o} from Sozin's Comet, Part 2: The Old Masters...
converting line {[Close-u} from Sozin's Comet, Part 2: The Old Masters...
converting line {[Close-u} from Sozin's Comet, Part 2: The Old Masters...
converting line {So, Bumi} from Sozin'

converting line {And I'm } from Sozin's Comet, Part 3: Into the Inferno...
converting line {But ...} from Sozin's Comet, Part 3: Into the Inferno...
converting line {Goodbye!} from Sozin's Comet, Part 3: Into the Inferno...
converting line {Please s} from Sozin's Comet, Part 3: Into the Inferno...
converting line {[Backed } from Sozin's Comet, Part 3: Into the Inferno...
converting line {That's h} from Sozin's Comet, Part 3: Into the Inferno...
converting line {Good wor} from Sozin's Comet, Part 3: Into the Inferno...
converting line {[Sarcast} from Sozin's Comet, Part 3: Into the Inferno...
converting line {I was ta} from Sozin's Comet, Part 3: Into the Inferno...
converting line {That wou} from Sozin's Comet, Part 3: Into the Inferno...
converting line {[Approac} from Sozin's Comet, Part 3: Into the Inferno...
converting line {Take us } from Sozin's Comet, Part 3: Into the Inferno...
converting line {[To a ne} from Sozin's Comet, Part 3: Into the Inferno...
converting line {Oh, hi. }

converting line {Well, lo} from Sozin's Comet, Part 4: Avatar Aang...
converting line {[Weakly.} from Sozin's Comet, Part 4: Avatar Aang...
converting line {[Mocking} from Sozin's Comet, Part 4: Avatar Aang...
converting line {[Patheic} from Sozin's Comet, Part 4: Avatar Aang...
converting line {[Pointin} from Sozin's Comet, Part 4: Avatar Aang...
converting line {You need} from Sozin's Comet, Part 4: Avatar Aang...
converting line {Mai! You} from Sozin's Comet, Part 4: Avatar Aang...
converting line {My uncle} from Sozin's Comet, Part 4: Avatar Aang...
converting line {So does } from Sozin's Comet, Part 4: Avatar Aang...
converting line {[Blushes} from Sozin's Comet, Part 4: Avatar Aang...
converting line {Dad! [Th} from Sozin's Comet, Part 4: Avatar Aang...
converting line {I heard } from Sozin's Comet, Part 4: Avatar Aang...
converting line {There's } from Sozin's Comet, Part 4: Avatar Aang...
converting line {[Appeari} from Sozin's Comet, Part 4: Avatar Aang...
converting line {[Pr

### Okay, that's enough sentiment data collection for now! Let's turn the data we've collected into meaning!